In [1]:
# import numpy as np

# from toolboxes.inference_toolbox.parameter import Parameter
# from toolboxes.inference_toolbox.likelihood import Likelihood
# from toolboxes.inference_toolbox.model import Model

# I_y = Parameter(name=['I_y','I_z'],prior_select='gaussian',multi_mode=True).add_prior_param(
#     'mu',  [[0,0], [10,10], [20,20]]).add_prior_param(
#         'cov',[[[1,0],[0,1]],[[2,0],[0,2]],[[3,0],[0,3]]])
# I_y.sample_param()



In [2]:
import numpy as np
import pandas as pd
from toolboxes.inference_toolbox.model import Model, add_model, delete_model

add_model('full_log_gpm','log10(Q/(2*u*pi*I_y*I_z*(x-x_0)**2)*exp(-(y-y_0)**2/(2*I_y**2*(x-x_0)**2))*(exp(-(z-z_0)**2/(2*I_z**2*(x-x_0)**2))+exp(-(z+z_0)**2/(2*I_z**2*(x-x_0)**2))))',
#            all_param_names=['I_y','I_z','Q','H','u', 'x_0', 'y_0', 'z_0'], independent_variables=['x','y','z'], dependent_variables=['Concentration'])

    # delete_model('gpm_norm')
# add_model('curve', 'm*x**2+c',all_param_names=['m','c'], dependent_variables=['y'], independent_variables=['x'])
model = Model('full_log_gpm').add_fixed_model_param('H', 1).add_fixed_model_param('u', 1).add_fixed_model_param('x_0', 0).add_fixed_model_param('y_0', 1).add_fixed_model_param('z_0', 1)
model.get_model()(pd.Series({'I_y': 1, 'I_z':2, 'Q':3}), [np.array([1,2,3]), np.array([4,5,6]), np.array([7,8,9])])

In [3]:
# from toolboxes.data_processing_toolbox.raw_data_processor import RawDataProcessor

# import os

# os.chdir('/PhD_project/')

# processor_params = {
#     'experiments_list': [
#         'Exp1',
#         'Exp2',
#         'Exp3',
#         'Exp4',
#         'Exp5',
#         'Exp6',
#         'Exp7',
#         'Exp8',
#         'Exp9',
#         'Exp10',
#         'Exp11',
#         'Exp12',
#     ],
#     'meta_data_select': 'GBR_data_summary',
#     'input_header': 'Concentration',
#     'output_header': 'Concentration',
#     'log_output_data':True,
#     'gridding': [500,500,100]
# }

# train, test = RawDataProcessor('GBR_data', 'GBR_data_gridded_500_500_100', 'GBR_processor', processor_params).process_data()
# train

In [4]:
from toolboxes.inference_toolbox.model import Model
from toolboxes.inference_toolbox.likelihood import Likelihood
from toolboxes.inference_toolbox.parameter import Parameter
from toolboxes.plotting_toolbox.domain import Domain
from toolboxes.inference_toolbox.sampler import Sampler
from toolboxes.data_processing_toolbox.sim_data_processor import SimDataProcessor
import numpy as np
import pandas as pd
import os

os.chdir('/PhD_project/')



# Define your main process as a function
sim_model = Model('log_gpm_norm').add_fixed_model_param('H', 1).add_fixed_model_param('I_y',1).add_fixed_model_param('I_z',1).add_fixed_model_param('Q',1e13)
sim_domain = Domain('cone_from_source_z_limited').add_domain_param('r',1000).add_domain_param('theta',np.pi/8).add_domain_param('source', [0,0,1]).add_domain_param('resolution',100)
simulator_params = {
    'output_header': 'Concentration',
    'log_output_data': True,
}

sim_data_processor = SimDataProcessor('sim_data_2', sim_model, sim_domain, simulator_params)

model = Model('log_gpm_norm').add_fixed_model_param('H', 1)
likelihood = Likelihood('gaussian')
I_y_and_I_z = Parameter(name=['I_y','I_z'],prior_select='log_norm',multi_mode=True).add_prior_param('mu',  [[1,1], [1.1,1.1], [1.2,1.2]]).add_prior_param('cov',[[[1,0],[0,1]],[[2,0],[0,2]],[[3,0],[0,3]]])
Q = Parameter(name='Q',prior_select='log_norm').add_prior_param('mu',  1e13).add_prior_param('cov', 1e12)
sigma = Parameter(name='sigma',prior_select='log_norm').add_prior_param('mu',  1).add_prior_param('cov', 1)
inference_params = pd.Series({'I_y_and_I_z': I_y_and_I_z, 'Q': Q, 'sigma':sigma})

samples, chain_samples, fields = Sampler(inference_params, model, likelihood, sim_data_processor, n_samples = 100).sample_all()


/build/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data already simulated, loading data...


sample: 100%|██████████| 150/150 [00:34<00:00,  4.29it/s, 511 steps of size 1.22e-13. acc. prob=0.43] 


In [5]:
from toolboxes.inference_toolbox.model import Model
import inspect

sim_model = Model('log_gpm_norm').add_fixed_model_param('H', 1).add_fixed_model_param('I_z',1).add_fixed_model_param('Q',1e13)

# Define some example parameters and independent variables
example_params = {'I_y': 3.0}
example_independent_vars = {'x': [1, 2, 3,1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9], 'y': [4, 5, 6,1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9], 'z': [7, 8, 9,1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9]}

# Get the model function
model_func = sim_model.get_model()

import inspect
lines = inspect.getsource(sim_model.expr_func)
print(lines)

def _lambdifygenerated(x, y, z, I_y):
    return log(5000000000000.0*(exp(-0.5*(z + 1.0)**2/x**2) + exp(-0.5*(z - 1.0)**2/x**2))*exp(-1/2*y**2/(I_y**2*x**2))/(pi*I_y*x**2))/log(10)

